In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pandas as pd
from bramm_data_analysis import preprocessing
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstools as gs

verbose = True
graphics = True
gl.OptCst.define(gl.ECst.NTCOL, 6)
gdoc.setNoScroll()

In [ ]:
moss_data_path = Path("../data/Mines_2024.xlsx")
df = preprocessing.MossPreprocessor(moss_data_path).load_preprocess()

In [ ]:
# Remove Columns with nans
df.loc[
    df["tree_layer_complement"].isna(), "tree_layer_complement"
] = "unspecified"
print("The following columns contains NaN and will be removed :")
print(df.columns[df.isna().any(axis=0)])
df = df.drop(columns=df.columns[df.isna().any(axis=0)])

In [ ]:
df.columns

In [ ]:
x1 = "longitude"
x2 = "latitude"
z1 = "iron"
z2 = "aluminium"
raw_df = df.filter([x1, x2, z1, z2]).astype("float")
final_df = raw_df[(raw_df[z2] <= 8000) & (raw_df[z1] <= 4000)]

In [ ]:
mydb = gl.Db_fromPanda(final_df)
mydb.setLocators([x1, x2], gl.ELoc.X)
mydb.setLocator(z1, gl.ELoc.Z)

In [ ]:
mydb

In [ ]:
gl.dbStatisticsPrint(
    db=mydb,
    names=[z1, z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=False,
    title="Number of observations",
)

In [ ]:
ax = gp.correlation(mydb, namex=z1, namey=z2, asPoint=True, regrLine=True)
ax.decoration(title=f"Correlation between {z1} and {z2}")

In [ ]:
varioparam = gl.VarioParam.createMultiple(
    ndir=2, npas=30, dpas=0.2, toldis=0.6
)
vario_raw2dir = gl.Vario(varioparam)
err = vario_raw2dir.compute(mydb)

In [ ]:
gp.setDefault(dims=[6, 6])
gp.varmod(vario_raw2dir)
plt.show()

In [ ]:
fitmod_raw = gl.Model()
err = fitmod_raw.fit(
    vario_raw2dir,
    types=[gl.ECov.NUGGET, gl.ECov.EXPONENTIAL, gl.ECov.CUBIC, gl.ECov.LINEAR],
)
fitmod_raw.setDriftIRF(0, 0)

In [ ]:
ax = gp.varmod(vario_raw2dir, fitmod_raw)
ax.decoration(
    title=f"Experimental and fitted variogram models - Raw {z2} observations",
    xlabel="Distance (km)",
    ylabel="Variogram",
)